# [Create Agent With OpenAPI](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-openapi)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_openapi.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py)

# Constants

In [1]:
import os, jsonref
from dotenv import load_dotenv # requires python-dotenv
from common.agents_helper_functions import *

# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
deployment_name = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] #  the type must be "gpt-4o-2024-08-06": https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...hub01-grp;mmai-swc-hub01-prj01>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails # <<<<<<<<<<<<<<< SPECIFIC FOR OPENAPI
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai-swc-hub01-grp',
 'project_name': 'mmai-swc-hub01-prj01'}

# Load Weather OpenAPI

In [3]:
# https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/weather_openapi.json
with open("./openapi/weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_spec

{'openapi': '3.1.0',
 'info': {'title': 'get weather data',
  'description': 'Retrieves current weather data for a location based on wttr.in.',
  'version': 'v1.0.0'},
 'servers': [{'url': 'https://wttr.in'}],
 'auth': [],
 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location',
    'operationId': 'GetCurrentWeather',
    'parameters': [{'name': 'location',
      'in': 'path',
      'description': 'City or location to retrieve the weather for',
      'required': True,
      'schema': {'type': 'string'}},
     {'name': 'format',
      'in': 'query',
      'description': 'Always use j1 value for this parameter',
      'required': True,
      'schema': {'type': 'string', 'default': 'j1'}}],
    'responses': {'200': {'description': 'Successful response',
      'content': {'text/plain': {'schema': {'type': 'string'}}}},
     '404': {'description': 'Location not found'}},
    'deprecated': False}}},
 'components': {'schemes': {}}}

# Create Auth object for the OpenApiTool 
## Note: connection or managed identity auth setup requires additional setup in Azure

In [4]:
auth = OpenApiAnonymousAuthDetails()
auth

{'type': 'anonymous'}

# Initialize agent OpenApi tool using the read in OpenAPI spec

In [5]:
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)
print(f"openapi.definitions: {openapi.definitions}")
print(f"\nopenapi.resources: {openapi.resources}")

openapi.definitions: [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema': {'type': 'string', 'default': 'j1'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'text/plain': {'schema': {'type': 'string'}}}}, '404': {'description': 'Location not found'}}, 'deprecated': 

# Create AI Foundry Agent

In [6]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=deployment_name,
    name="openapi-agent",
    instructions="You are a helpful assistant",
    tools=openapi.definitions,
    tool_resources = openapi.resources,
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_t4b4fM7l4oKIR76kJBXkzyQc', 'object': 'assistant', 'created_at': 1745764284, 'name': 'openapi-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 

# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_18JufbILe3s7HTCNWVHzvhrz', 'object': 'thread', 'created_at': 1745764284, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_WddQ3v7nQzJrKtFuBoUX2pvb', 'object': 'thread.message', 'created_at': 1745764285, 'assistant_id': None, 'thread_id': 'thread_18JufbILe3s7HTCNWVHzvhrz', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent synchronously

In [8]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_rMIydREVDx530jV5xDWCG2Uq', 'object': 'thread.run', 'created_at': 1745764286, 'assistant_id': 'asst_t4b4fM7l4oKIR76kJBXkzyQc', 'thread_id': 'thread_18JufbILe3s7HTCNWVHzvhrz', 'status': 'completed', 'started_at': 1745764286, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1745764288, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'l

# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: Non è possibile prevedere la temperatura del 15 giugno, poiché i dati meteo sono disponibili solo fino al 29 aprile 2025. 

Per quel giorno (29 aprile), la temperatura massima prevista a Chiavari è di **18°C**.


# Run Steps

In [10]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_PbEtoLxa8mBpaFTiQThqStBT', 'object': 'thread.run.step', 'created_at': 1745764288, 'run_id': 'run_rMIydREVDx530jV5xDWCG2Uq', 'assistant_id': 'asst_t4b4fM7l4oKIR76kJBXkzyQc', 'thread_id': 'thread_18JufbILe3s7HTCNWVHzvhrz', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1745764288, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_dcvNXfrpEsE9psd6EIc8buvr'}}, 'usage': {'prompt_tokens': 9301, 'completion_tokens': 56, 'total_tokens': 9357, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_Tt0gBEAY3diy4O6AkoNzscxv', 'object': 'thread.run.step', 'created_at': 1745764287, 'run_id': 'run_rMIydREVDx530jV5xDWCG2Uq', 'assistant_id': 'asst_t4b4fM7l4oKIR76kJBXkzyQc', 'thread_id': 'thread_18JufbILe3s7HTCNWVHzvhrz', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 

# Collect all resources for this project

In [11]:
all_agents = list_all_agents(project_client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(project_client)
print(all_threads["summary"])

all_files = list_all_files(project_client)
print(all_files["summary"])

all_runs = list_all_runs(project_client)
print(all_runs["summary"])

all_runsteps=list_all_runsteps(project_client)
print(all_runsteps["summary"])

all_messages = list_all_messages(project_client)
print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(project_client=project_client)
print(all_vectorstores["summary"])

1 agents in project <mmai-swc-hub01-prj01>
1 threads in project <mmai-swc-hub01-prj01>
0 files in project <mmai-swc-hub01-prj01>
1 runs in 1 threads of project <mmai-swc-hub01-prj01>
2 run steps in 1 pairs of (thread, run) of project <mmai-swc-hub01-prj01>
2 messages in 1 threads of project <mmai-swc-hub01-prj01>
0 vector stores in project <mmai-swc-hub01-prj01>


# Teardown for all resources

In [12]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]["data"]:
    i += 1
    project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(project_client=project_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [13]:
# delete all files

i=0
for file in all_files['content']['data']:
    i += 1
    project_client.agents.delete_file(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [14]:
# delete all threads

i=0
for thread in all_threads["content"]["data"]:
    i += 1
    project_client.agents.delete_thread(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_18JufbILe3s7HTCNWVHzvhrz> has been deleted
Threads deleted: 1



In [15]:
# delete all agents

i=0
for agent in all_agents["content"]["data"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(project_client=project_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_t4b4fM7l4oKIR76kJBXkzyQc> has been deleted
Agents deleted: 1



# HIC SUNT LEONES